In [1]:
%run base_test.ipynb

import numpy as np

from base import ml
from base import plot

In [2]:
type_ = 'grid'
average = True
poly = True
bilinear = True

if type_ == 'point':
    dir_ = '../saved_models/point'
    gen_one_data = ml.GenSolutionPoint(fenics_from_save=True)
    PolyInterp = ml.PolyInterpPoint
    BilinearInterp = ml.BilinearInterpPoint
elif type_ == 'grid':
    dir_ = '../saved_models/grid'
    gen_one_data = ml.GenSolutionGrid(fenics_from_save=True)
    PolyInterp = ml.PolyInterpGrid
    BilinearInterp = ml.BilinearInterpGrid

In [3]:
dnn_factories, names = ml.dnn_factories_from_dir(dir_)
extra_facs = []
extra_names = []

In [4]:
if average:
    average_reg = ml.RegressorAverager(regressor_factories=dnn_factories)
    average_reg.auto_mask(gen_one_data=gen_one_data, top=5, batch_size=10000)
    average_reg_fac = ml.RegressorFactory(regressor=average_reg)
    extra_facs.append(average_reg_fac)
    extra_names.append('AverageReg')

INFO:tensorflow:Using config: {'_model_dir': '../saved_models/grid/128x10_121__ScaleCenter_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa448164d68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../saved_models/grid/128x10_121__ScaleCenter_crelu/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Using

In [5]:
if poly:
    # Warning: the polynomial interpolation takes a while.
    for poly_deg in (3, 5, 7, 9):
        poly_fac = ml.RegressorFactory(regressor=PolyInterp(poly_deg=poly_deg))
        extra_facs.append(poly_fac)
        extra_names.append('Poly-{}'.format(poly_deg))

In [6]:
if bilinear:
    bilin_fac = ml.RegressorFactory(regressor=BilinearInterp())
    extra_facs.append(bilin_fac)
    extra_names.append('BilinearReg')

In [7]:
results = ml.eval_regressors([*dnn_factories,
                              *extra_facs],
                             [*names,
                              *extra_names]
                             gen_one_data, 
                             10000)

INFO:tensorflow:Using config: {'_model_dir': '../saved_models/grid/128x10_121__ScaleCenter_crelu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa4441b21d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../saved_models/grid/128x10_121__ScaleCenter_crelu/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Using

KeyboardInterrupt: 

In [ ]:
bests = []
for stat in ('loss', 'average_loss', 'max_deviation'):
    print(stat)
    print('-' * len(stat))
    results_names = sorted(zip(results, [*names, *extra_names]), 
                               key=lambda x: x[0][stat])
    bests.append([name for _, (result, name) in zip(range(6), results_names)])
    for result, name in results_names:
        try:
            num_neurons = eval(name.split('_')[0].replace('x', '*'))
        except Exception:
            num_neurons = ''
        print('{:>4} | {:>34} | {}'.format(num_neurons, 
                                           name, 
                                           result[stat]))
    print('')
    
best = [x for x in bests[0] if all(x in best_ for best_ in bests[1:])]
print('common best')
print('-----------')
for b in best:
    print(b)
if not best:
    print('No common best')